## Python imports

In [ ]:
from jinja2 import Template
import os
import sys

### _Load Django into Jupyter_

In [ ]:
sys.path.append('/bookstore')
sys.path.append('/bookstore/project')
os.environ['DJANGO_SETTINGS_MODULE'] = 'project.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'
import django
django.setup()

## Adding LogLevels Objects

In [ ]:
with open("/templates/dbfill_bookstore.py", "r+") as file:
    data = file.read()
    if not os.path.exists("/bookstore/bookstore/scripts"):
        os.makedirs("/bookstore/bookstore/scripts")
    with open("/bookstore/bookstore/scripts/dbfill_bookstore.py", "w") as file:
        write_file = file.write(data)

print(open("/bookstore/bookstore/scripts/dbfill_bookstore.py").read())

In [ ]:
%%bash

cd /bookstore
python manage.py runscript dbfill_bookstore

### _let's check . . ._

In [ ]:
from bookstore.models import *

for b in Book.objects.all():
    print("%s, written by %s" % (b,b.author))

## Configuring Serialization

In [ ]:
file = '/bookstore/bookstore/serializers.py'
    
with open(file, 'w') as file:
    template = Template(open('/templates/serializers.j2', 'r').read())
    print(template.render())
    file.write(template.render())

## Challenge #3
### Explain how serialization helps with data collection

In [ ]:
from bookstore.serializers import *

author = Author.objects.last()
author.__dict__
# print(dir(author))
# print(dir(author.changed_by))

print(f"Updated by {author.changed_by.first_name} {author.changed_by.last_name}")

AuthorDetailSerializer(author).data


In [ ]:
book = Book.objects.last()
book.title = f"{book.title} - Second Edition"
book.changeReason = "New edition"
book.changed_by_id = 2
book.__dict__
# book.save()

In [ ]:
from django.core import serializers
import json


serialized_obj = json.loads(serializers.serialize('json', [book, ]))[0]
serialized_obj['history'] = []
for h in book.history.all():
    sh = json.loads(serializers.serialize('json', [h, ]))[0]
    serialized_obj['history'].append(sh)

print(json.dumps(serialized_obj, indent=4, sort_keys=True))